# Jumbled Words

<!-- ### I had to start somewhere...

It's a personal hobby of mine to spend a little time each day devoted to learning new things. 
For some time now, I have been meaning to get in the habit of documenting and publishing my findings, 
both to cement concepts I've learned and perhaps to help someone else who may be doing some learning of their own.
This isn't by any means the first or the flashiest project I've worked on, but I had to start somewhere. Why not with something simple?
 -->

## Motivation

I recently came across an article talking about how people can read jumbled text, given that the first and last letters of each word remain in their respective positions. 
The article included a few examples of jumbled sentences, and I found the concept to be intriguing. Unsatisfied with example jumbled text from the article, I resolved to explore this phenomenon further. Since the format for jumbling words is both clearly defined and simple, I figured it could be the basis of a simple mini-project to brush up on my python skills. I decided to write a simple program that can take a text snippet and output a jumbled variation of it.

### Implementation

First and foremost we should clearly define the rules of the jumbling algorithm that we want to implement, and I have done so in the short list below:

#### Rules of Jumbling

1. The first and last letter of each word must remain in place
2. The letters in the middle of each word can be mixed up randomly in any order
3. By default, words that are 3 letters or less remain unchanged. 

For the sake of simplicity, I wrote my program to ignore the puncuation and capitalization of the text that it jumbles. Doing this greatly simplifies the implementation at the cost of making the jumbled result slightly more difficult to read. Using a little extra brain power never hurt anyone, right?

After doing some brief research, I chose to import a few packages to help with the processes performed in the jumbling algorithm. 
The ```random``` library will make it nice and easy to jumble the letters, and the ```re``` library helps to strip the punctuation from the imput text. Base python should be equipped with everything else we need, so let's get started on piecing things together. We'll also be using the ```argparse``` library to make our code usable from the command line.

I've recently come to learn that drawing a picture is a great way to start most coding projects as it provides a tentative roadmap of what needs to happen and how the different steps can be broken up. This program is simple enough that an actual sketch might not be necessary, so a simple list should suffice:

#### Word Jumble Program Steps:

1. Read in text from a file
2. Format the text by removing punctuation and capitalization
3. Jumble the text according to the algorithm definition
4. Save the jumbled text to a new file

Keeping these steps in mind, we will proceed with the implementation. At the end, we'll join things up into a single script that we can run from the command line.


Importing libraries is easy enough, and we already discussed why these libraries in particular will be useful for this program.

In [1]:
# libraries needed for the jumbling program
import random
import re
import argparse

Next, we can use a simple context manager to read in the text from a file. 

In [2]:
# read in the text from a file
with open('example.txt') as f:
    content = f.read()
print(content)

Here is an example of some words that can be jumbled up. As long as we don't mess with the first letter or the last letter, we can read it without any problems. Kind of cool, right?


The jumbling algorithm is simple enough that it can be coded into a single function, which I implemented below.
I tried to be thorough in terms of commenting each step, but I also included a more detailed description of how the function works in the text that follows.

In [3]:
# function to perform the jumbling algorithm
def jumble(content):
    
    # remove punctuation and make lower case
    content = re.sub('[^\w\s]','',content).lower() 
    # split up the string based on spaces
    content_list = content.split()  
    
    #initialize output string and use a for loop to update it
    new_content = ""
    for word in content_list:
        if len(word) > 3:
            # isolate first and last letters of the word
            first_letter = word[0]
            last_letter = word[-1]
            
            # extract the middle of the word and jumble it up!
            middle = word[1:-1]            
            jumbled_middle = random.sample(middle, len(middle))
            
            # reassemble the parts of the jumbled word
            jumbled_word = first_letter + "".join(jumbled_middle) + last_letter
        else:
            # the word is too short to be jumbled
            jumbled_word = word
        # update the output
        new_content += jumbled_word + " "
    
    return new_content

**Step 1: Format the content string**

The content argument is simply a string of the text that we want to jumble.
The ```sub``` function from the ```re``` library is how we strip the punctuation. 
The first argument is the pattern that we want to replace: ```'[^\w\s]'``` (this  regular expression isolates punctuation).
The second argument is what we want to replace any matches with. 
Since we want to remove punctuation marks, we just replace them with the empty string: ```''```. 
The third and final argument of the ```sub``` function is the string from in which to look for matches, which, 
in this case, is the content string that we passed into the jumble function.
To handle capitalization, we used the ```.lower()``` method from base python to make the entire content string lower case.
Lastly, we used the ```.split()``` method (also from base python) to split up the content string into a list of words which will be jumbled in the next step.

**Step 2: Jumble the words**

Now that the content string has been split up into a list of words, we can simply loop through the list,
and jumble it according to the rules that we defined earlier. 
If the word is 3 letters or less we simply add it to the output string, otherwise we jumble it!
When jumbling, we make use of the ```sample``` function from the ```random``` library. 
The ```sample``` function takes two arguments: a population from which to sample, and the number (k) of samples to draw from that population. 
Since strings in python are treated essentially as an array of characters we can pass in ```middle``` as the population 
arguement where ```middle``` is the array remaining characters of each word after excluding the first and last characters.
For k, we simply pass in the length of ```middle``` since we want to get each of the characters returned back to us 
in a random order.
The returned list of individual characters is connected back together using the ```.join()``` method (base python), and the 
first/last letters are placed at there proper positions, leaving us with our jumbled word.
After adding this word to the output string the loop restarts, repeating the jumbling process until completion.

Let's put our new function to the test and see if it is working properly.

In [4]:
# show original content
print("Original Content:\n", content,"\n", sep = "")

# jumble the content and show the results!
jumbled_content = jumble(content)
print("Jumbled Content:\n", jumbled_content, sep = "")

Original Content:
Here is an example of some words that can be jumbled up. As long as we don't mess with the first letter or the last letter, we can read it without any problems. Kind of cool, right?

Jumbled Content:
hree is an exlmpae of some wdros that can be jubmled up as long as we dnot mess wtih the first leettr or the last letetr we can raed it woiutht any peombrls knid of cool rhigt 


Seems to be working great!

Next we'll compile each of the pieces together into a single script and use the ```argparse``` library to make a program that's easy to run from the command line. 

```argparse``` is pretty easy to work with. We just need to initialize a parsing object, specify the arguments that we want to recieve, and then parse the received arguments. This can be done in just a few lines of code. I chose to call my arguments ```-i``` and ```-o``` for input and output respectively. I used the ```dest``` argument to provide a more accurate name when extracting them from the parser. 

```python
# parse the arguments from the command line
parser = argparse.ArgumentParser(description='This is a program that jumbles text')
parser.add_argument("-i", help="The name of the file with text to jumble", dest="input_file")
parser.add_argument("-o", help="The name of the output file that containt the jumbled text",
                    dest="output_file", default="output.txt")
args = parser.parse_args()
```

The```description``` and  ```help``` arguments are used in the program help text which can be seen if we run the program from the command line with the ```-h``` argument.

![](helpmessage.png)

Now that we have the argparse section complete we can throw the individual parts together into a single script.

The final result looks like this:

```python
# jumble.py --------------------------------------------------------------

# libraries
import argparse
import re
import random


def main():
    # parse the arguments from the command line
    parser = argparse.ArgumentParser(description='This is a program that jumbles text')
    parser.add_argument("-i", help="The name of the file with text to jumble", dest="input_file")
    parser.add_argument("-o", help="The name of the output file that containt the jumbled text",
                        dest="output_file", default="output.txt")
    args = parser.parse_args()

    # extract args from arg parser
    input_file = args.input_file
    output_file = args.output_file

    # read in the text from a file
    with open(input_file) as f:
        text = f.read()

    # jumble the text from input_file
    output = jumble(text)

    # write the output to a new file
    with open(output_file, 'w') as f:
        f.write(output)

    # display message
    print("The text from {0} was jumbled and saved to {1}".format(input_file, output_file))
    return


# function to perform the jumbling algorithm
def jumble(content):
    # remove punctuation and make lower case
    content = re.sub('[^\w\s]', '', content).lower()
    # split up the string based on spaces
    content_list = content.split()

    # initialize output string and use a for loop to update it
    new_content = ""
    for word in content_list:
        if len(word) > 3:
            # isolate first and last letters of the word
            first_letter = word[0]
            last_letter = word[-1]

            # extract the middle of the word and jumble it up!
            middle = word[1:-1]
            jumbled_middle = random.sample(middle, len(middle))

            # reassemble the parts of the jumbled word
            jumbled_word = first_letter + "".join(jumbled_middle) + last_letter
        else:
            # the word is too short to be jumbled
            jumbled_word = word
        # update the output
        new_content += jumbled_word + " "
    return new_content


if __name__ == "__main__":
    main()
```

Running the script, we can see that it works!
<br>
![](workingversion.png)


Thanks for reading this through to the end!
I hope you found it helpful.
If you have any questions or feedback, feel free to reach out to me on [LinkedIn](https://www.linkedin.com/in/mattoehler/).
All of the code for this project can be found on my [github]().

Happy Coding :)